# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [24]:
nutrition_agent = Agent(
    name="Nutrition agent",
    instructions="""
    you are helpul
    you give concise answers
    """
)

In [28]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition agent", ...)
- Final output (str):
    Bananas are quite healthy and convenient. Key points:
    
    - Nutrients: good source of potassium (~400 mg), vitamin B6, vitamin C, and dietary fiber.
    - Calories: about 90–105 per medium banana; mostly natural sugars and carbs.
    - Benefits: supports heart health, digestion, and immune function; low fat and gluten-free.
    - Considerations: higher sugar content than some fruits; portion if you’re managing blood sugar.
    - Other: inexpensive, portable, and widely available.
    
    Overall: a healthy, nutritious snack when eaten in typical portions.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [25]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Overall, yes—bananas are healthy as part of a balanced diet.

Key nutrients (per medium banana, ~118 g):
- Potassium: supports blood pressure and heart function
- Vitamin B6, vitamin C, fiber
- Some magnesium and manganese

Calories: about 90; sugars ~14 g; carbs ~23 g; low sodium

Benefits:
- Quick energy from natural sugars
- Supports digestion (fiber)
- May aid heart health and blood sugar regulation when eaten in moderation

Notes:
- Riper bananas have more sugar; unripe have more resistant starch
- Portion control for diabetes or high sugar concerns
- Easy to add to meals (snack, cereal, smoothies)

Bottom line: healthy, versatile, build them into a varied diet.

_Good Job!_